In [1]:
import pandas as pd

In [2]:
ds = pd.read_csv("Social_Network_Ads.csv")

In [3]:
ds=pd.get_dummies(ds,drop_first=True)

In [4]:
ds.drop("User ID",axis=1) #axis 1 Column, 0 Row

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [5]:
ds['Purchased'].value_counts() # to check balance or inbalanced data set - inbalanced 

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
independent = ds[['Age', 'EstimatedSalary','Gender_Male']]
dependent = ds[['Purchased']]

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=False)

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = { 
            'criterion' : ['gini', 'entropy', 'log_loss'],
             'splitter' : ['best', 'random'],
            'max_features' : ['auto','sqrt','log2']
                     }

Classifier = GridSearchCV(DecisionTreeClassifier(),param_grid, refit = True, scoring='f1_weighted', n_jobs=-1, verbose = 3)

# Creats the model
Classifier.fit(x_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\hitov\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
30 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
17 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\hitov\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hitov\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\hitov\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\hitov\anaconda3\Lib\site-packages\

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [20]:
result = Classifier.cv_results_

In [21]:
y_pred = Classifier.predict(x_test)

In [22]:
# Built the matrix
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,y_pred)
print(matrix)

[[77  8]
 [ 6 43]]


In [23]:
# fetch the Classification report
from sklearn.metrics import classification_report
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.91      0.92        85
           1       0.84      0.88      0.86        49

    accuracy                           0.90       134
   macro avg       0.89      0.89      0.89       134
weighted avg       0.90      0.90      0.90       134



In [24]:
# Key to built the model
from sklearn.metrics import f1_score
f1_macro = f1_score (y_test,y_pred,average='weighted')
print ("The best value for Parameter {}:".format(Classifier.best_params_),f1_macro)


The best value for Parameter {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'best'}: 0.8959452736318407


In [25]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score (y_test,Classifier.predict_proba(x_test)[:,1]) # : tell all rows, take 1 ist colum of all rows for probability 
print ("The best value for Parameter {}:".format(Classifier.best_params_),roc_auc)
  

The best value for Parameter {'criterion': 'gini', 'max_features': 'log2', 'splitter': 'best'}: 0.8917166866746699


In [26]:
table = pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005810,0.003016,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
1,0.004743,0.004629,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
2,0.022216,0.006312,0.041712,0.015524,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.782557,0.868752,0.796284,0.851527,0.831098,0.826044,0.032483,9
3,0.020612,0.001760,0.046144,0.006263,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.765716,0.906252,0.832483,0.829124,0.831098,0.832935,0.044524,6
4,0.017914,0.003765,0.049482,0.009089,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.829615,0.885035,0.775815,0.867097,0.924528,0.856418,0.050571,1
5,0.020901,0.002813,0.043737,0.002453,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.787654,0.868752,0.814409,0.849057,0.831098,0.830194,0.027936,8
6,0.003522,0.002911,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
7,0.004824,0.001488,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
8,0.015864,0.002564,0.049339,0.004404,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.804764,0.831253,0.773585,0.813179,0.841025,0.812761,0.023408,12
9,0.020302,0.003559,0.043954,0.004175,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.849794,0.850809,0.796284,0.844370,0.883278,0.844907,0.027913,3


In [ ]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

In [ ]:
Classifier.predict([[age_input,salary_input,sex_male_input]])